Add subtype info

In [1]:
## Metadata
METADATA_RPATH <- "data/yeoh_2002/processed/metadata.tsv"
metadata <- read.table(METADATA_RPATH, header = T, sep = "\t",
                        comment.char = "", row.names = 1)
## Add subtype info
class_info <- as.factor(substring(rownames(metadata), 1, 1))
levels(class_info) <- c("Hyperdiploid", "E2A-PBX1", "TEL-AML1",
                        "BCR-ABL", "Hypodiploid", "Pseudodiploid",
                        "MLL", "T-ALL", "Normal")
edited_metadata <- cbind(metadata, class_info)

METADATA_WPATH <- "data/yeoh_2002/processed/metadata1.tsv"
write.table(edited_metadata, METADATA_WPATH,
            quote = F, sep = "\t")

Further separate hyperdiploid into subtypes (>50) and (47-50) and add relapse info

In [34]:
## Metadata
METADATA1_RPATH <- "data/yeoh_2002/processed/metadata1.tsv"
metadata1 <- read.table(METADATA1_RPATH, header = T, sep = "\t",
                        comment.char = "", row.names = 1)

In [ ]:
## Distinguish hypdip (>50) from (47-50)
levels(metadata1$class_info)[3] <- "Hyperdiploid (>50)"
levels(metadata1$class_info) <- c(levels(metadata1$class_info), "Hyperdiploid (47-50)")
metadata1$class_info[65:87] <- "Hyperdiploid (47-50)"

In [68]:
split_info <- strsplit(as.character(metadata1$original_names), split = "-")
raw_label <- sapply(split_info,
                    function(x) ifelse(x[length(x)] != "N",
                                       x[length(x)], x[length(x)-1]))
map_label <- function(x) {
  if (startsWith(x, "C")) {
    return("CCR")
  } else if (startsWith(x, "R")) {
    return("Heme relapse")
  } else if (startsWith(x, "2")) {
    return("2nd AML")
  } else if (startsWith(x, "#")) {
    return(NA)
  } else {
    stop("No matching substring!")
  }
}
                    
outcome <- sapply(raw_label, map_label)

In [79]:
metadata2 <- cbind(metadata1, outcome)

## Manual editing of relapse info according to suppl data
subset_pid <- c("T-ALL-#1", "T-ALL-#2", "TEL-AML1-#1", "Pseudodip-#1")
idx <- which(metadata2$original_names %in% subset_pid)
levels(metadata2$outcome) <- c(levels(metadata2$outcome), "Other relapse")
metadata2$outcome[idx] <- "Other relapse"

## Binarise outcome into label relapse: ("")
label <- ifelse(metadata2$outcome == "CCR", 0, 1)
metadata3 <- cbind(metadata2, label)

METADATA_WPATH <- "data/yeoh_2002/processed/metadata2.tsv"
write.table(metadata3, METADATA_WPATH,
            quote = F, sep = "\t")